# All imports necessary

In [ ]:
import sys

In [ ]:
sys.path.append('..')

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
from source.code.utils import filter_by_subcorpus
from source.code.utils import get_tagged_texts_as_pd

In [ ]:
from source.code.preprocessing import filtrations
from source.code.preprocessing import additional_features
from source.code.preprocessing import crf_filtration_and_pre_processing

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

In [ ]:
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics

# Read the data

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
tagged_texts_as_pd = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
tagged_texts_as_pd_f = filtrations(tagged_texts_as_pd)

In [ ]:
tagged_texts_as_pd_f_add_f = additional_features(tagged_texts_as_pd_f)

In [ ]:
tagged_texts_as_pd_f_add_f.info()

In [ ]:
features = ['semantic_relation_tagged', 'animacy_tagged', 'lambda_dsr_len', 'word_sense_exists', 'is_title', 'contains_digits', 'word_len']
target = 'ner_tag'

# Naive tag frequency memorization

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {}
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [ ]:
X, y = tagged_texts_as_pd_f_add_f.token, tagged_texts_as_pd_f_add_f.ner_tag

In [ ]:
pred = cross_val_predict(estimator=MemoryTagger(), X=X, y=y, cv=5)

In [ ]:
report = classification_report(y_pred=pred, y_true=y)
print(report)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X, y = tagged_texts_as_pd_f_add_f[features], tagged_texts_as_pd_f_add_f[target]

In [ ]:
pred = cross_val_predict(RandomForestClassifier(n_estimators=20), X=X, y=y, cv=5)

In [ ]:
report = classification_report(y_pred=pred, y_true=y)
print(report)

# HMM

# CRF

In [ ]:
from sklearn_crfsuite import CRF

In [ ]:
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)

In [ ]:
tagged_texts_as_pd_f = filtrations(tagged_texts_as_pd, with_dots=True)

In [ ]:
tagged_texts_as_pd_add_f = additional_features(tagged_texts_as_pd_f)

In [ ]:
sentences, tags = crf_filtration_and_pre_processing(tagged_texts_as_pd_add_f)

In [ ]:
pred = cross_val_predict(crf, X=sentences, y=tags, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=tags)
print(report)

# Bi-LSTM

In [ ]:
tagged_texts_as_pd_f = filtrations(tagged_texts_as_pd, with_dots=True)

In [ ]:
tagged_texts_as_pd_add_f = additional_features(tagged_texts_as_pd_f)

In [ ]:
sentences, tags = crf_filtration_and_pre_processing(tagged_texts_as_pd_add_f)

In [ ]:
words = list(set([word['lemma'] for sentence in sentences for word in sentence]))
words.append("ENDPAD")
n_words = len(words); n_words

In [ ]:
unique_tags = tagged_texts_as_pd_f.ner_tag.unique()
n_tags = len(unique_tags); n_tags

In [ ]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(unique_tags)}

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w['lemma']] for w in s] for s in sentences]

In [ ]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [ ]:
y = [[tag2idx[w] for w in s] for s in tags]

In [ ]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [ ]:
from keras.utils import to_categorical

In [ ]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print(flat_classification_report(pred_labels, test_labels))

In [ ]:
class BiLSTMTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        
    
    def predict(self, X, y=None):
        

# Conclusion